# Electrospray Thruster Modeling

In [80]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import pickle
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox
import ipywidgets as widgets
import time
from numpy.random import normal
import os
import datetime
from IPython.display import set_matplotlib_formats
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy import optimize
set_matplotlib_formats('pdf', 'svg')
mpl.rcParams['figure.dpi'] = 300

## 1. Monte Carlo Analysis

In [2]:
from Capillary_IL import Model

In [3]:
fname = 'x.pkl'
input_data = pickle.load(open(fname, 'rb'))
emi_bf4 = pickle.load(open('EMI_BF4.pkl', 'rb'))

In [4]:
def hydro_resist(r, L, mu):
    '''Calculate the hydraulic resistance of the feed'''
    return (8/np.pi/r**4)*mu*L

In [5]:
def set_props(BV, CR_limit, CR_m, CL, CR, div_mult, DEL, N, p_scale, TED, Feed_R, Feed_L, prop):

    global input_data
    input_data['Boost Voltage'] = BV
    input_data['CR Limit'] = CR_limit
    input_data['CR_m'] = CR_m
    input_data['Capillary Length'] = CL
    input_data['Capillary Radius'] = CR
    input_data['Div Mult'] = div_mult
    input_data['Droplet Energy Loss'] = DEL
    input_data['Ne'] = N
    input_data['P-Scale'] = p_scale
    input_data['Tip-to-Extractor Distance'] = TED
    input_data['feed_resist'] = 0

    if Feed_R != 0 and Feed_L != 0:
        print('here')
        if prop == 'EMI-BF4':
            input_data['feed_resist'] = hydro_resist(
                Feed_R, Feed_L, emi_bf4['Dynamic_Viscosity'])
        else:
            input_data['feed_resist'] = hydro_resist(
                Feed_R, Feed_L, input_data['Propellants']['Dynamic_Viscosity'])

In [6]:
def do_calc(data, prop, volt_range, calc_div, divs, **kwargs):
    '''Perform the electrospray calculations'''
    data['Capillary Radius_std'] = 0
    data['V'] = np.linspace(volt_range[0], volt_range[1], divs)
#     data['P'] = np.array([2.5])
    data['Feed Resistance'] = np.ones(data['V'].shape) * input_data['feed_resist']
    data['new_div'] = calc_div
    if prop == 'EMI-BF4':
        data['Propellants'] = emi_bf4
    y = Model.fields(data)[0]
    df = pd.DataFrame.from_dict(y)
    df['Propellant'] = prop
    return df

In [7]:
def run_sim(x, y, prop, volt_range, mc, num_runs, new_run, calc_div, divisions, show_both, switch=False, **kwargs):
    '''Run the simulation with or without Monte Carlo'''

    if not mc:
        if prop == 'Both':
            df1 = do_calc(input_data.copy(), 'EMI-BF4',
                          volt_range, calc_div, divisions)
            df2 = do_calc(input_data.copy(), 'EMI-TFSI',
                          volt_range, calc_div, divisions)
            if show_both:
                df1['Divergence Calc'] = calc_div
                df2['Divergence Calc'] = calc_div
                df3 = do_calc(input_data.copy(), 'EMI-BF4',
                          volt_range, not calc_div, divisions)
                df4 = do_calc(input_data.copy(), 'EMI-TFSI',
                          volt_range, not calc_div, divisions)
                df1['Divergence Calc'] = not calc_div
                df2['Divergence Calc'] = not calc_div
                
                df = pd.concat([df1, df2, df3, df4])
            else: 
                df = pd.concat([df1, df2])
        else:
            if show_both:
                df1 = do_calc(input_data.copy(), prop,
                         volt_range, calc_div, divisions)
                df1['Divergence Calc'] = calc_div
                df2 = do_calc(input_data.copy(), prop,
                         volt_range, not calc_div, divisions)
                df2['Divergence Calc'] = not calc_div

                df = pd.concat([df1, df2])
            else:
                df = do_calc(input_data.copy(), prop,
                         volt_range, calc_div, divisions)
              
        if show_both:
            sns.relplot(x=x, y=y, data=df, kind='line', hue='Propellant', style='Divergence Calc')
        else:
            sns.relplot(x=x, y=y, data=df, kind='line', hue='Propellant')
        plt.xlabel(x+' ['+Model.UNITS[x]+']')
        plt.ylabel(y+' ['+Model.UNITS[y]+']')
        plt.show()
    else:
        if new_run:
            w = widgets.FloatProgress()
            w.min = 0
            w.max = num_runs-1
            w.description = 'Running:'
            display(w)
            if 'cap_rad' in kwargs:
                cap_rad = kwargs['cap_rad']
            else:
                cap_rad = input_data['Capillary Radius']
            if 'cap_std' in kwargs:
                cap_std = kwargs['cap_std']
            else:
                cap_std = 0.1*cap_rad

            cap_rads = normal(cap_rad, cap_std, num_runs)
            if prop == 'Both':
                df1 = do_calc(input_data.copy(), 'EMI-BF4',
                              volt_range, calc_div, divisions, cap_rad=cap_rads[0])
                df2 = do_calc(input_data.copy(), 'EMI-TFSI',
                              volt_range, calc_div, divisions, cap_rad=cap_rads[0])
                df = pd.concat([df1, df2])
            else:
                df1 = do_calc(input_data.copy(), prop,
                              volt_range, calc_div, divisions, cap_rad=cap_rads[0])
                df = df1

            for i in np.arange(1, num_runs):
                if prop == 'Both':
                    df1 = do_calc(input_data.copy(), 'EMI-BF4',
                                  volt_range, calc_div, divisions, cap_rad=cap_rads[i])
                    df2 = do_calc(input_data.copy(), 'EMI-TFSI',
                                  volt_range, calc_div, divisions, cap_rad=cap_rads[i])
                    df = pd.concat([df, df1, df2])
                else:
                    df1 = do_calc(input_data.copy(), prop,
                                  volt_range, calc_div, divisions, cap_rad=cap_rads[i])
                    df = pd.concat([df, df1])
                w.value = i
                w.description = str(
                    np.around(100*i/(num_runs-1), decimals=1)) + ' %'
            w.description = 'Done!'

            fname = 'mc_' + prop + '_' + str(calc_div) + '.p'
            pickle.dump(df, open('./monte_carlo_output_data/' + fname, 'wb'))
            sns.relplot(x=x, y=y, data=df, kind='line',
                        hue='Propellant', ci='sd')
            plt.xlabel(x+' ['+Model.UNITS[x]+']')
            plt.ylabel(y+' ['+Model.UNITS[y]+']')
            plt.show()
        else:
            fname = 'mc_' + prop + '_' + str(calc_div) + '.p'
            try:
                df_mc = pickle.load(
                    open('./monte_carlo_output_data/' + fname, 'rb'))
            except FileNotFoundError:
                print('File not found. Please perform a new run.')
                return

            sns.relplot(x=x, y=y, data=df_mc,
                        kind='line', hue='Propellant', ci='sd')
#             plt.xlim(volt_range)
            plt.xlabel(x+' ['+Model.UNITS[x]+']')
            plt.ylabel(y+' ['+Model.UNITS[y]+']')
            plt.show()

In [8]:
style = {'description_width': 'initial'}
x_widget = widgets.Dropdown(options=['V', 'Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current', 'Divergence Angle'],
                            value='V', description='x value', style=style)
y_widget = widgets.Dropdown(options=['V', 'Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current', 'Divergence Angle'],
                            value='Isp', description='y value', style=style)
volt_widget = widgets.IntRangeSlider(min=0, max=10000, step=100, continuous_update=False,
                                     description='Voltage Range', style=style)
num_run_widget = widgets.IntSlider(min=0, max=1000, value=500, step=100, continuous_update=False,
                                   description='Number of Runs', style=style)
mc_widget = widgets.Checkbox(
    value=False, description='Run Monte Carlo', style=style)
prop_widget = widgets.Dropdown(options=['EMI-TFSI', 'EMI-BF4', 'Both'], value='EMI-TFSI',
                               description='Propellant', style=style)
new_run_widget = widgets.Checkbox(
    value=False, description='New Monte Carlo Run', style=style)
cap_rad_widget = widgets.BoundedFloatText(value=3.95E-6, description='Capillary Radius',
                                          style=style)
cap_rad_std_widget = widgets.BoundedFloatText(value=2.5E-7, description='Capillary Radius St.D',
                                              style=style)
use_calc_div_widget = widgets.Checkbox(
    value=True, description='Adjust with Calc Divergence', style=style)
divisions_widget = widgets.IntSlider(
    min=101, max=5001, description='Voltage Divisons', style=style)

show_both_div_widget = widgets.Checkbox(
    value=False, description='Show Calculation with and Without Divergence Calc', style=style)

BV = widgets.FloatText(value=0, description='Boost Voltage', style=style)
CR_limit = widgets.FloatText(value=3, description='CR Limit', style=style)
CR_m = widgets.FloatText(
    value=1.806e-8, description='Slope Parameter', style=style)
CL = widgets.FloatText(
    value=100e-05, description='Capillary Length', style=style)
CR = widgets.FloatText(
    value=74e-06, description='Capillary Radius', style=style)
div_mult = widgets.FloatText(
    value=23, description='Divergence Multiplier', style=style)
DEL = widgets.FloatText(
    value=0, description='Droplet Energy Loss', style=style)
N = widgets.IntText(value=1, description='Number of Emitters', style=style)
p_scale = widgets.FloatText(value=0, description='P_scale', style=style)
TED = widgets.FloatText(
    value=0.0003, description='Tip-to-Extractor Distance', style=style)
Feed_R = widgets.FloatText(
    value=7.6e-5, description='Feed Radius', style=style)
Feed_L = widgets.FloatText(value=1.016, description='Feed Length', style=style)


Lbox = VBox([x_widget, y_widget, prop_widget, volt_widget,
             num_run_widget, divisions_widget, mc_widget, new_run_widget, use_calc_div_widget, show_both_div_widget])
Cbox = VBox([cap_rad_widget, cap_rad_std_widget])
Rbox = VBox([BV, CR_limit, CR_m, CL, CR, div_mult,
             DEL, N, p_scale, TED, Feed_R, Feed_L])
ui = HBox([Lbox, Rbox])

props = widgets.interactive_output(set_props, {'BV': BV, 'CR_limit': CR_limit, 'CR_m': CR_m, 'CL': CL,
                                               'CR': CR, 'div_mult': div_mult, 'DEL': DEL, 'N': N,
                                               'p_scale': p_scale, 'TED': TED, 'Feed_R': Feed_R, 'Feed_L': Feed_L,
                                               'prop': prop_widget})
sim = widgets.interactive_output(run_sim, {'x': x_widget, 'y': y_widget, 'prop': prop_widget, 'mc': mc_widget,
                                           'volt_range': volt_widget, 'num_runs': num_run_widget,
                                           'new_run': new_run_widget, 'cap_rad': cap_rad_widget,
                                           'cap_std': cap_rad_std_widget, 'calc_div': use_calc_div_widget,
                                           'divisions': divisions_widget, 'show_both': show_both_div_widget})
display(ui, sim)

Output()

## 2. Sensitivity Analysis

This section aims to identify the influence of the variance of input parameters on the output of the model. Saltelli sampling with Sobol sensisitivity analysis is implemented. The user can choose the number of samples to choose as well as specify the bounds for the input parameters.

There are 11 main factors that influence the output of the simulation, namely:

* Boost Voltage - Acceleration voltage in addition to extraction voltage
* CR Limit - Propellant specific mode switching parameter. At dimensionless hydrualic resistances CR greater than CR Limit, emission site is modeled to operate in pure ionic mode
* CR_m - VI slope parameter for pure ionic mode
* Capillary Length
* Capillary Radius
* ~~Divergence Angle - Spray divergence half angle~~
* Divergence Multiplier - Constant that is experimentally evaluated
* Droplet Energy Loss - Estimated energy loss of droplets due to cone-jet electrical resistance
* P scale - Scaling factor for field-induced pressure
* Tip-to-Extractor Distance
* Feed Length
* Feed Radius

In [9]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [10]:
BV = [0, 2500]
CR_limit = [0, 3]
CR_m = [0, 2e-8]
CL = [80e-5, 120e-5]
CR = [65e-6, 85e-6]
#DA = [5, 40]
# div_mult = [1, 100]
P = [0, 100]
DEL = [0, 200]
p_scale = [0, 1]
TED = [0.0001, 0.001]
Feed_R = [1.3e-4, 1.7e-4]
Feed_L = [1, 1.03]

names = ['Boost Voltage', 'CR Limit', 'CR_m', 'Capillary Length',
         'Capillary Radius', 'P', 'Droplet Energy Loss',
         'P-Scale', 'Tip-to-Extractor Distance', 'Feed Radius', 'Feed Length']
bounds = [BV, CR_limit, CR_m, CL, CR, P, DEL, p_scale, TED, Feed_R, Feed_L]
problem = {'num_vars': 11, 'names': names, 'bounds': bounds}

In [11]:
Si = {}


def print_out(s):
    print('Analyzing: ' + s + '...', end='')


def sensitivity(N, Save_Data):

    param_values = saltelli.sample(problem, N)
    Y = np.zeros([param_values.shape[0]])
    x = input_data.copy()
    V_len = 101
    Vs = np.linspace(500, 3000, V_len)
    x['V'] = Vs
    x['Ne'] = 1
    w = widgets.FloatProgress(style=style)
    out = widgets.Output(layout={'border': '1px solid black'})
    w.min = 0
    w.max = len(param_values)
    display(w)
    global Si

    model_runs = []
    x['new_div'] = True
    mu = x['Propellants']['Dynamic_Viscosity']

    for i, vals in enumerate(param_values):
        x['Feed Resistance'] *= 0
        for field, val in zip(names, vals):
            x[field] = val
            if field == 'P':
                x[field] = np.array([val])
        x['Feed Resistance'] += hydro_resist(x['Feed Radius'],
                                             x['Feed Length'], mu)
        y_temp = Model.fields(x)[0]
        w.value += 1
        w.description = 'Running Simulations...' + \
            str(np.around(100*w.value/len(param_values), decimals=1)) + ' %'
        model_runs.append(y_temp)

    w.description = 'Done!'
    for F in ['Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current', 'Divergence Angle']:

        print_out(F)

        sub_probs = []
        for i in np.arange(0, V_len):
            j = 0
            for m in model_runs:
                Y[j] = m[F][i]
                j += 1
            temp_res = sobol.analyze(problem, Y)
            sub_probs.append(temp_res)
        Si[F] = sub_probs

        print('Done!')

    if Save_Data:
        pickle.dump(
            Si, open('./sensitivity_output_data/sens_out_'+str(N)+'_all_V_P.p', 'wb'))
        pickle.dump(model_runs, open(
            './sensitivity_output_data/model_runs_'+str(N)+'_all_V_P.p', 'wb'))

In [12]:
sens_sim = interact_manual(sensitivity, N=(1, 5000), Save_Data=False)
sens_sim.widget.children[2].description = 'Run Sensitivity'

interactive(children=(IntSlider(value=2500, description='N', max=5000, min=1), Checkbox(value=False, descripti…

In [97]:
sns.set()
fnames = []
for root, dirs, files in os.walk('./sensitivity_output_data'):
    for file in files:
        if file.endswith('.p'):
            fnames.append(root+'/'+file)


@interact
def parametric_sensitivty(field=['Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current', 'Divergence Angle'],
                          order=['S1', 'S2', 'ST'], plot_type=['Bar', 'Pie'],
                          file=widgets.Select(
                              options=fnames, description='Files'),
                          robust=True,
                          png=False):

    if png:
        set_matplotlib_formats('png')
    else:
        set_matplotlib_formats('pdf', 'svg')

    try:
        Si = pickle.load(open(file, 'rb'))
    except FileNotFoundError:
        print('Sensitivity analysis not performed')
        return

    V_len = 101
    Vs = np.linspace(500, 3000, V_len)
    if order != 'S2':
        data = [Si[field][i][order] for i in np.arange(0, len(Si[field]))]
        data_array = np.nan_to_num(np.array(data))
        df = pd.DataFrame(data_array, columns=names)
        p_scale = df['P-Scale']
        df['V'] = Vs
        df = df.melt('V', var_name='Param', value_name=order)

        sns.relplot(x='V', y=order, data=df, kind='line', hue='Param')
        
        def test_func(x, a, b):
            return a*x**b
        
        params, params_covariance = optimize.curve_fit(test_func, Vs[60:], p_scale[60:], p0=[-1, 2])
        print(params)
        
        
        
#         for i in np.arange(0, data_array.shape[1]):
#             #             if names[i] == 'Tip-to-Extractor Distance':
#             #                 continue
#             plt.plot(Vs, data_array[:, i], label=names[i])

#         plt.legend(bbox_to_anchor=(1, 1))
        plt.xlim((1000, 3000))
        plt.ylim((-0.25, 1.25))
#         plt.ylabel(order)
#         plt.xlabel('Voltage [V]')
        plt.title(field)
        plt.show()

interactive(children=(Dropdown(description='field', options=('Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Elec…

In [14]:
sns.set()
fnames = []
for root, dirs, files in os.walk('./sensitivity_output_data'):
    for file in files:
        if file.endswith('.p'):
            fnames.append(root+'/'+file)


@interact
def visual_sens(field=['Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current', 'Divergence Angle'],
                order=['S1', 'S2', 'ST'], plot_type=['Bar', 'Pie'],
                file=widgets.Select(options=fnames, description='Files'),
                robust=True,
                png=False):

    if png:
        set_matplotlib_formats('png')
    else:
        set_matplotlib_formats('pdf', 'svg')

    try:
        Si = pickle.load(open(file, 'rb'))
    except FileNotFoundError:
        print('Sensitivity analysis not performed')
        return
    temp_dat = Si[field][50]
    data = temp_dat[order]
    ind = np.arange(len(data))
    if order != 'S2':
        if plot_type == 'Bar':
            plt.bar(ind, data, yerr=temp_dat[order+'_conf'])
            plt.xticks(ind, names, rotation='vertical')
            plt.ylim(top=1.5)
            plt.ylim(bottom=-0.1)
        elif plot_type == 'Pie':
            plt.pie(data, labels=names, autopct='%1.1f%%', startangle=90)
            
        plt.ylabel(order)
        plt.title(field)
        plt.show()
    else:
        #         data = temp_dat[order+'_conf']
        df = pd.DataFrame(100*np.transpose(data), columns=names)
        sns.heatmap(df, yticklabels=names, annot=True,
                    fmt='.1f', robust=robust)
        plt.title(field + ', S2 (x100)')
        plt.show()

interactive(children=(Dropdown(description='field', options=('Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Elec…

### Monte Carlo from Sensitivity Data

In [15]:
fnames = []
for root, dirs, files in os.walk('./sensitivity_output_data'):
    for file in files:
        if file.endswith('.p'):
            fnames.append(root+'/'+file)

@interact_manual
def load_mc_data(file=widgets.Select(options=fnames, description='Files')):

    global plot_frame
    try:
        plot_data = pickle.load(open(file, 'rb'))
        data_frames = []
        for i in np.arange(0, len(plot_data)):
            temp_df = pd.DataFrame.from_dict(plot_data[i])
            data_frames.append(temp_df)
        
        plot_frame = pd.concat(data_frames)
        print('Loaded data successfully.')
    except FileNotFoundError:
        print('Invalid file selection')

interactive(children=(Select(description='Files', options=('./sensitivity_output_data/model_runs_100.p', './se…

In [108]:
def tsplot(x, y, n=20, percentile_min=1, percentile_max=99, color='b', plot_mean=True, plot_median=False, line_color='k', **kwargs):
    # calculate the lower and upper percentile groups, skipping 50 percentile

    percs1 = np.linspace(percentile_min, 50, num=n, endpoint=False)
    percs2 = np.linspace(50, percentile_max, num=n+1)[1:]
    perc1 = np.percentile(y, percs1, axis=0)
    perc2 = np.percentile(y, percs2, axis=0)

    if 'alpha' in kwargs:
        alpha = kwargs.pop('alpha')
    else:
        alpha = 1/n
    # fill lower and upper percentile groups
    for p1, p2 in zip(perc1, perc2):
        plt.fill_between(x, p1, p2, alpha=alpha, color=color, edgecolor=None)

    if plot_mean:
        plt.plot(x, np.mean(y, axis=0), color=line_color)

    if plot_median:
        plt.plot(x, np.median(y, axis=0), color=line_color)

    return plt.gca()


@interact_manual
def sens_MC(x=['V', 'Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current', 'Divergence Angle'],
            y=['V', 'Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Electric Current',
                'Divergence Angle', 'Median CR', 'Number of Sites In Ion Mode', 'Number of Active Sites'],
            png=False):

    global plot_frame
    if png:
        set_matplotlib_formats('png')
    else:
        set_matplotlib_formats('pdf', 'svg')

    x_data = plot_frame[x][:101]
    y_data = plot_frame[y]

    y_data = y_data.values.reshape((-1, 101))


#     sns.relplot(x=x, y=y, data=plot_frame, ci='sd', kind='line')
#     plt.scatter(plot_x, plot_y)
    tsplot(x_data, y_data, percentile_min=25, percentile_max=75,
           n=1, plot_mean=False, plot_median=True, alpha=0.3)
    tsplot(x_data, y_data, percentile_min=5, percentile_max=95,
           n=1, plot_mean=False, plot_median=False, alpha=0.3)

#     sns.scatterplot(x=x, y=y, data=plot_frame, estimator=None, lw=1)
    plt.title(x + ' vs. ' + y)
#     plt.xlim((1000, 3000))
    plt.xlabel(x+' ['+Model.UNITS[x]+']')
    plt.ylabel(y+' ['+Model.UNITS[y]+']')
    plt.show()

interactive(children=(Dropdown(description='x', options=('V', 'Isp', 'Thrust', 'Mass Flow', 'Efficiency', 'Ele…